## GPR with Derivatives

A Gaussian process regression is used for mapping from input variables to output variables.

In some cases, it can be useful to model not just the output variables, but also their derivatives. 

Gaussian process with derivatives (GPD) is a variant of GRP which models both output variables and their derivatives.

GPD properties:

If $f \sim GP(\mu, k)$, then
$$
\nabla f \sim GP(\frac{\partial}{\partial x} \mu, \frac{\partial^2}{\partial x \partial y}{k})
$$

In words, if our original model of $f$ is drawn from a GP with a certain mean and kernel function, then the gradient of our model $f$ is drawn from a GP with a related mean and kernel.

#### Take an Example of 1D input

Suppose
$$
\mu(x) = 0
$$
and
$$
k(x, y) = \exp(-.5 (x-y)^2/l^2)
$$
where the number of dimensions is $1$ for simplicity.

Then
$$
\nabla f \sim GP(\frac{d}{dx} \mu, \frac{d^2}{d x d y}{k})
$$

where

1.
$$
\frac{d}{d x} \mu = \begin{pmatrix}
0 \\
\vdots \\
0 \\
\end{pmatrix}
$$
2.
$$
\frac{d}{d x} k(x, y) = -\frac{x - y}{l^2}k(x, y)
$$
3. 
$$
\frac{d^2}{dx dy} k(x, y) = \frac{1}{l^2}(1 - (x - y)^2/l^2)k(x, y)
$$

In [13]:
import cupy as cp
import matplotlib.pyplot as plt

class GaussianRegressionWithDerivatives():
    def __init__(self, kernel=None, dd_kernel=None, sigma=1e-6):
        self.kernel = kernel
        self.dd_kernel = dd_kernel
        self.sigma = sigma
        self.X = None
        self.y = None
        
    def fit(self, X, y, dy):
        self.X = X
        self.y = y
        self.dy = dy
        # print('the shape of self.kernel(X, X) is: ' + str(self.kernel(X, X).shape))
        self.K = self.kernel(X, X) + self.sigma**2 * cp.eye(len(X))
        # print('the shape of self.K is: ' + str(self.K.shape))
        self.dK = self.dd_kernel(X, X) + self.sigma**2 * cp.eye(len(X))
        self.L = cp.linalg.cholesky(self.K)
        self.dL = cp.linalg.cholesky(self.dK)
        # let alpha = inv(K) * y => K * alpha = y => L*L.T * alpha = y => alpha = solve(L.T, solve(L, y))
        self.alpha = cp.linalg.solve(self.L.T, cp.linalg.solve(self.L, y))
        self.dalpha = cp.linalg.solve(self.dL.T, cp.linalg.solve(self.dL, dy))

        return self
        
    def predict(self, Xtest):
        K_train_test = self.kernel(self.X, Xtest)
        y_pred = K_train_test.T @ self.alpha

        dK_train_test = self.dd_kernel(self.X, Xtest)
        dy_pred = dK_train_test.T @ self.dalpha

        # L*v = K => v = inv(L)*K => v.T * v = K.T * inv(L.T) * inv(L) * K = K.T * inv(L*L.T) * K
        # v = cp.linalg.solve(self.L, K_train_test)
        # cov = self.kernel(Xtest, Xtest) - v.T @ v
        # std = cp.sqrt(cp.diag(cov))
        # return mean, cov, std
        return y_pred, dy_pred

def d_mu(x):
    return 0

def RBF(x1, x2, length_scale=1.0):
    diff_mat = x1[:, cp.newaxis] - x2
    return cp.exp(-0.5 * diff_mat ** 2 / length_scale ** 2)

def d_RBF(x1, x2, length_scale=1.0):
    diff_mat = x1[:, cp.newaxis] - x2
    return -diff_mat * cp.exp(-0.5 * diff_mat ** 2 / length_scale ** 2) / length_scale ** 2

def dd_RBF(x1, x2, length_scale=1.0):
    diff_mat = x1[:, cp.newaxis] - x2
    return (1 - diff_mat**2/length_scale**2)/length_scale**2 * cp.exp(-0.5 * diff_mat ** 2 / length_scale ** 2)

def RMSE(y, y_pred):
    return cp.sqrt(cp.mean((y_pred - y) ** 2))
    
X = cp.array(cp.random.uniform(-5, 5, size=(50,)))
y = cp.array(cp.sin(X) + cp.random.normal(0, 0.1, size=(50,)))
dy = cp.array(cp.cos(X))

gr = GaussianRegressionWithDerivatives(RBF, dd_RBF)
gr.fit(X,y,dy)

# use the same data set as training, y_pred and dy_pred should be similar to y and dy
y_pred, dy_pred = gr.predict(X)

y_error = RMSE(cp.array(y), cp.array(y_pred))
dy_error = RMSE(cp.array(dy), cp.array(dy_pred))

'''
print('y is: \n' + str(y[:10]))
print('y_pred is: \n' + str(y_pred[:10]))
print('dy is: \n' + str(dy[:10]))
print('dy_pred: \n' + str(dy_pred[:10]))
'''

print('RSME for y_error is: ' + str(y_error))
print('RSME for dy_error is: ' + str(dy_error))

RSME for y_error is: 0.05160168445709523
RSME for dy_error is: 2.0927974227329497e-08


### Above example missed the cross kernel parts, and it works only when we feed derivatives to the model
Modity it to get a more general GPD model:
$$
\begin{pmatrix}
f \\
\nabla f
\end{pmatrix} \sim GP\left(
\begin{pmatrix}
\mu(x) \\
\frac{d}{dx} \mu(x)
\end{pmatrix}
, 
\begin{pmatrix}
k(x, y) & \frac{d}{dy}k(x, y)^T \\
\frac{d}{dx}k(x, y) & \frac{d^2}{d x d y}{k}(x, y) \\
\end{pmatrix}
\right)
$$

Use jax for following implementations since jax can return gradients (numpy and cupy don't)

In [39]:
## Validating my d_RBF() and dd_RBF
import jax
import numpy as np
import jax.numpy as jnp
from jax import jit, grad
from jax.scipy.linalg import solve_triangular
from functools import partial

def RBF(x1, x2, length_scale=1.0):
    # The reason we write it this way is because the second derivative
    # when x=y does not seem to work ...
    diff_mat = x1[:, jnp.newaxis] - x2
    return jnp.exp(-0.5 * diff_mat ** 2 / length_scale ** 2)

def d_RBF(x1, x2, length_scale=1.0):
    diff_mat = x1[:, jnp.newaxis] - x2
    return -diff_mat * jnp.exp(-0.5 * diff_mat ** 2 / length_scale ** 2) / length_scale ** 2

def dd_RBF(x1, x2, length_scale=1.0):
    diff_mat = x1[:, jnp.newaxis] - x2
    return (1 - diff_mat**2/length_scale**2)/length_scale**2 * jnp.exp(-0.5 * diff_mat ** 2 / length_scale ** 2)

def RMSE(y, y_pred):
    return jnp.sqrt(jnp.mean((y_pred - y) ** 2))

# use jax.jexrev() and jax.jacfwd() to validate my own d_RBF() and dd_RBF()
jax_dx = jax.jacrev(RBF, argnums=0)
jax_dx_dy = jax.jacfwd(jax.jacrev(RBF, argnums=0), argnums=1)


x1 = jnp.array([1.,2.,3.])
x2 = jnp.array([1.,2.,3.])

kernel_dx = -d_RBF(x1, x2)
jax_kernel_dx = np.sum(jax_dx(x1, x2), axis=0)

kernel_dx_dy = dd_RBF(x1, x2)
jax_kernel_dx_dy = np.sum(np.sum(jax_dx_dy(x1, x2), axis=0), axis=0)

print("kernel_dx from d_RBF(): \n" + str(kernel_dx))
print("kernel_dx from jax.jacrev(): \n" + str(jax_kernel_dx))

print("kernel_dx_dy from d_RBF(): \n" + str(kernel_dx_dy))
print("kernel_dx_dy from jax.jacrev(): \n" + str(jax_kernel_dx_dy))



kernel_dx from d_RBF(): 
[[ 0.         -0.60653067 -0.27067056]
 [ 0.60653067  0.         -0.60653067]
 [ 0.27067056  0.60653067  0.        ]]
kernel_dx from jax.jacrev(): 
[[ 0.         -0.60653067 -0.27067056]
 [ 0.60653067  0.         -0.60653067]
 [ 0.27067056  0.60653067  0.        ]]
kernel_dx_dy from d_RBF(): 
[[ 1.          0.         -0.40600586]
 [ 0.          1.          0.        ]
 [-0.40600586  0.          1.        ]]
kernel_dx_dy from jax.jacrev(): 
[[ 1.          0.         -0.40600586]
 [ 0.          1.          0.        ]
 [-0.40600586  0.          1.        ]]


In [46]:
def kernal_derivative(x1, x2):
    kern = RBF(x1, x2)
    left = d_RBF(x1, x2)
    right = -d_RBF(x1, x2)
    hessian = dd_RBF(x1, x2)

    # Put everything together
    top = jnp.concatenate([kern, right], axis=1)
    bot = jnp.concatenate([left, hessian], axis=1)
    K = jnp.concatenate([top, bot])
    return K

K = kernal_derivative(x1,x2)
print(K)

[[ 1.          0.60653067  0.13533528  0.         -0.60653067 -0.27067056]
 [ 0.60653067  1.          0.60653067  0.60653067  0.         -0.60653067]
 [ 0.13533528  0.60653067  1.          0.27067056  0.60653067  0.        ]
 [-0.          0.60653067  0.27067056  1.          0.         -0.40600586]
 [-0.60653067 -0.          0.60653067  0.          1.          0.        ]
 [-0.27067056 -0.60653067 -0.         -0.40600586  0.          1.        ]]


Fit a GP to both data and derivatives:
$$
\begin{pmatrix}
f \\
\nabla f
\end{pmatrix} = 
\begin{pmatrix}
\mu(x) \\
\frac{d}{dx} \mu(x)
\end{pmatrix} + 

\begin{pmatrix}
k*(x, y) & \frac{d}{dy}k*(x, y)^T \\
\frac{d}{dx}k*(x, y) & \frac{d^2}{d x d y}{k*}(x, y) \\
\end{pmatrix} * 

\begin{pmatrix}
k(x, y) & \frac{d}{dy}k(x, y)^T \\
\frac{d}{dx}k(x, y) & \frac{d^2}{d x d y}{k}(x, y) \\
\end{pmatrix} ^{-1} * 

\begin{pmatrix}
f - mu(x)\\
f - dmu(x)
\end{pmatrix}

$$

Fit a GP to derivatives:
 
$$ \hat{y} = \frac{\partial}{\partial x}\mu(X_*) + K_{*X} (K_{XX} - \sigma^2 I)^{-1} (y_{X} - \frac{\partial}{\partial x}\mu(X)) $$

In [71]:
class gprDerivatives():
    def __init__(self, kernel=None, sigma=1e-6):
        self.kernel = kernel
        self.sigma = sigma
        self.X = None
        self.y = None
        
    def fit(self, X, y):
        self.X = X
        self.y = np.concatenate([y, y])
        self.K = kernal_derivative(X, X)
        self.L = np.linalg.cholesky(self.K)
        self.alpha = np.linalg.solve(self.L.T, np.linalg.solve(self.L, self.y))
        '''
        print('the shape of self.K is: ' + str(self.K.shape))
        print('the shape of self.y is: ' + str(self.y.shape))
        print('the shape of self.L is: ' + str(self.L.shape))
        print('the shape of self.alpha is: ' + str(self.alpha.shape))
        '''
        # let alpha = inv(K) * y => K * alpha = y => L*L.T * alpha = y => alpha = solve(L.T, solve(L, y))
        return self
        
    def predict(self, Xtest):
        K_train_test = kernal_derivative(X, Xtest)
        n = Xtest.shape[0]
        res_total = K_train_test.T @ self.alpha
        y_pred = res_total[:n]
        dy_pred =  res_total[n:]

        # L*v = K => v = inv(L)*K => v.T * v = K.T * inv(L.T) * inv(L) * K = K.T * inv(L*L.T) * K
        # v = cp.linalg.solve(self.L, K_train_test)
        # cov = self.kernel(Xtest, Xtest) - v.T @ v
        # std = cp.sqrt(cp.diag(cov))
        # return mean, cov, std
        return y_pred, dy_pred

X = np.array([1.,2.,3.])
y = np.array([1.,4.,9.])
Xtest = np.array([4.,5.])

gpd = gprDerivatives(kernel=RBF)
gpd.fit(X,y)
y_pred, dy_pred = gpd.predict(Xtest)

print(y_pred, dy_pred)


the shape of res_total is: (4,)
[18.60653  10.713235] [  2.9019544 -13.275953 ]
